In [1]:
import os

folder_pwd = os.getcwd()
os.chdir('..')
os.chdir('../resources')
resources_path = os.getcwd()
os.chdir(folder_pwd)

image_folder = "/test/Images"
csv_folder = '/test/c'

files = os.listdir(resources_path+csv_folder)
csv_file = []
for f in files:
    csv_file.append(f)

csv_file

['Batch10.csv - TEST.csv', 'Batch11.csv TEST.csv', 'Batch9.csv - TEST.csv']

In [2]:
%%time
from csv import reader
import pandas as pd
import numpy as np

def read_csv(file_name):
    return pd.read_csv(resources_path+csv_folder+'/'+file_name, names=['type','from_x','from_y','width', 'height', 'name', 'full_width', 'full_height'])

df = pd.DataFrame()
for f in csv_file:
    df = df.append(read_csv(f)).reset_index(drop=True)
df

Wall time: 348 ms


,type,from_x,from_y,width,height,name,full_width,full_height
0,Head,401,218,199,178,20170_2021-02-17_23.26_231242218.png,800,480
1,Head,414,190,194,173,20170_2021-02-17_23.26_23155437.png,800,480
2,Spine,360,121,165,76,20176_2021-02-18_14.12_1478842.png,800,480
3,Head,353,157,200,169,20180_2021-02-14_10.40_103946538.png,800,480
4,Head,429,141,133,125,20185_2021-02-19_14.07_14130658.png,800,480
...,...,...,...,...,...,...,...,...
432,Head,310,106,88,86,21273_2021-04-26_20.02_195224108.png,800,480
433,Hands,421,124,36,24,21273_2021-04-26_20.02_195224108.png,800,480
434,Head,256,101,109,102,21273_2021-04-26_20.13_20739369.png,800,480
435,Head,329,129,92,88,21277_2021-04-27_20.23_201955591.png,800,480


In [3]:
print(df['type'].unique())
df = df.rename(columns={'from_x':'from_y', 'from_y':'from_x'})
df = df.replace({'Head ':'head','Head':'head','Arms':'arm','Spine ':'spine','Legs':'leg','Feet':'foot','Spine':'spine', 'Hands':'hand'})
print(df['type'].unique(),'\n' ,df.groupby('type').count()['from_x'])
df

['Head ' 'Spine' 'Arms' 'Hands' 'Legs' 'Head']
['head' 'spine' 'arm' 'hand' 'leg'] 
 type
arm        7
hand      11
head     343
leg        2
spine     74
Name: from_x, dtype: int64


,type,from_y,from_x,width,height,name,full_width,full_height
0,head,401,218,199,178,20170_2021-02-17_23.26_231242218.png,800,480
1,head,414,190,194,173,20170_2021-02-17_23.26_23155437.png,800,480
2,spine,360,121,165,76,20176_2021-02-18_14.12_1478842.png,800,480
3,head,353,157,200,169,20180_2021-02-14_10.40_103946538.png,800,480
4,head,429,141,133,125,20185_2021-02-19_14.07_14130658.png,800,480
...,...,...,...,...,...,...,...,...
432,head,310,106,88,86,21273_2021-04-26_20.02_195224108.png,800,480
433,hand,421,124,36,24,21273_2021-04-26_20.02_195224108.png,800,480
434,head,256,101,109,102,21273_2021-04-26_20.13_20739369.png,800,480
435,head,329,129,92,88,21277_2021-04-27_20.23_201955591.png,800,480


In [4]:
images_to_check =df['name'].unique()
images_to_check.shape

(370,)

In [5]:
%%time

import os, random, cv2, Classify
import numpy as np

# 1. Load models
classifiers = Classify.load_svm(resources_path + '/SVM')
print(classifiers)

{'head': SVC(probability=True), 'spine': SVC(probability=True), 'leg': SVC(probability=True), 'arm': SVC(probability=True), 'foot': SVC(probability=True), 'hand': SVC(probability=True)}
Wall time: 1.04 s


In [ ]:
%%time
import time

results = []
for image in images_to_check:
    print(image, end=' ')
    img = cv2.imread(resources_path+image_folder+'/'+image, 0)
    
    start = time.time()

    feat_image, result, pred = Classify.analyse_image(img, classifiers,  crop_thresh=0.2, blur_size=10, bright_strength=1, sharp_strength=1,
                                   square_step=True, windows=[(64, 64), (96, 96), (128, 128), (192, 192)], window_stride=16, pred_threshold = 0.99)
    
    end = time.time()
    result['time_in_s'] = end-start
    result['name'] = image
    results.append(result)
    result.to_csv(resources_path+'/test/res/'+image[0:-4]+'.csv')

results = pd.concat(results)
results

20170_2021-02-17_23.26_231242218.png 20170_2021-02-17_23.26_23155437.png 20176_2021-02-18_14.12_1478842.png 

In [ ]:
results1 = results.rename(columns={'highest': 'type'}).drop(columns=['window', 'hog'])
results1 = pd.merge(df, results1, on=['name', 'type'], how='outer')
results1

In [ ]:
from Classify import iou_value

def iou(row):
    c, c1 = row[['from_x','from_y']], row[['x','y']]
    boxA = [c[0], c[1], c[0]+row['width'], c[1]+row['height']]
    boxB = [c1[0], c1[1], c1[0]+row['window_size'][0], c1[1]+row['window_size'][1]]
    return iou_value(boxA, boxB, boxes=True)

r = results1.dropna()
r.loc[:,'iou'] = r.apply(iou, axis=1)

true_positive = len(r[r['iou']>0.5])
false_positive = results1['x'].isna().sum()
false_negative = results1['from_x'].isna().sum()
print('{} tp {} fp {} fn'.format(true_positive, false_positive, false_negative))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(np.concatenate(results['window'].values, axis=1), cmap = 'gray')